In [8]:
from ollama_client import OllamaClient
from pydantic import BaseModel, Field
from tools import tavily_search
import datetime

print(datetime.datetime.now().strftime("%d %b %Y"))

24 Aug 2025


### Reformulate User Question

In [8]:
task = """you are an expert on simplifying user question for optimal google search. 
1. Identify 3 different  formulations of what the user is asking
2. keep the reformulations short and concise
3. the 3 formulations must be varied and must be useful and varied variations of the user query but must be strongly related to the user query
4. No need to write complete sentences but iinclude all important aspects"""
model1="qwen3:1.7b"
model2="llama3.2:3b"
model2="gemma3:4b"


re_phrase_agent = OllamaClient(role = task, model_name=model2)

class RePhraseResult(BaseModel):
    formulation_1: str
    formulation_2: str
    formulation_3: str


re_phrase_agent.invoke(query="""Hey Man. i love coding. 
                       I wanted to ask you several questions one of which is: How can I improve the performance of my Python code?""", json_schema=RePhraseResult)

{'text': '{"formulation_1": "Python code optimization", "formulation_2": "Python performance tuning", "formulation_3": "Speed up Python code"}',
 'raw': {'model': 'gemma3:4b',
  'created_at': '2025-08-24T09:52:19.008493023Z',
  'message': {'role': 'assistant',
   'content': '{"formulation_1": "Python code optimization", "formulation_2": "Python performance tuning", "formulation_3": "Speed up Python code"}'},
  'done_reason': 'stop',
  'done': True,
  'total_duration': 1124796766,
  'load_duration': 72588535,
  'prompt_eval_count': 367,
  'prompt_eval_duration': 239230355,
  'eval_count': 35,
  'eval_duration': 672319820},
 'conversation_history': [{'role': 'user',
   'content': 'Hey Man. i love coding. \n                       I wanted to ask you several questions one of which is: How can I improve the performance of my Python code?'},
  {'role': 'assistant',
   'content': '{"formulation_1": "Python code optimization", "formulation_2": "Python performance tuning", "formulation_3": "Spe

In [3]:
print(re_phrase_agent.system_instructions)

You are a helpful Agent among a group of agents trying to solve a problem. Each agent is tasked with a part or the entire problem.
You will be given your task. You will have access to all the relevant information and tools. You can also see the work already done by other agents. Use that information if required.


## Instructions
1. View the context, the task executed, the results, the tool call results of the other agents.
2. Reason and execute your task.
3. You have access to multiple tools as listed. You can only call tools which are relevant to your task. Other agents might have executed other tools which you dont have access to.
4. Always output strictly JSON. Always.
5. your task will be enclosed in <YOUR TASK></YOUR TASK> tags. This is your task. Only execute this task.
6. The work done by other agents will be enclosed in <Agent: Agent Name></Agent: Agent Name> tags. There may be multiple of these.


Following is the relevant information from other agents (if any):



<YOUR TASK

### Web Search Agent

In [ ]:
task = """You are an expert on simplifying user question for optimal google search and then invoking search tool. 
1. Identify 3 different formulations of what the user is asking
2. keep the reformulations short and concise. 
3. Invoke the tavily_search if it will help provide an updated and more accurate answer to the question.
4. If invoking the tool, extract from the user question 3 different ways of formulating the same question and pass to the search tool.
5. If the question needs to refer to documentation, do a web search and answer
6. If the user asks for recent events or asks to refer to the web, then use this tool
7. Rephrase the user question in 3 ways and pass each of them to the tool
"""
model1="mistral-small3.2:latest"

re_phrase_agent = OllamaClient(role = task, model_name=model1, agent_name="SearchAgent")

class AnswerBasedOnContext(BaseModel):
    answer: str

search_queries = re_phrase_agent.invoke(query="""Hey Man. i love coding. 
                       I wanted to ask you several questions one of which is: What is the latest version of gpt out there?""", tools=[tavily_search])

search_queries

{'text': '',
 'raw': {'model': 'mistral-small3.2:latest',
  'created_at': '2025-08-24T15:01:13.617996643Z',
  'message': {'role': 'assistant',
   'content': '',
   'tool_calls': [{'function': {'name': 'tavily_search',
      'arguments': {'query1': 'latest version of gpt',
       'query2': 'most recent version of gpt',
       'query3': 'current version of gpt'}}}]},
  'done_reason': 'stop',
  'done': True,
  'total_duration': 46688723851,
  'load_duration': 51536986,
  'prompt_eval_count': 806,
  'prompt_eval_duration': 39895268384,
  'eval_count': 41,
  'eval_duration': 6710347998},
 'conversation_history': [{'role': 'user',
   'content': 'Hey Man. i love coding. \n                       I wanted to ask you several questions one of which is: What is the latest version of gpt out there?'},
  {'role': 'assistant', 'content': ''}],
 'tool_calls': [{'function': {'name': 'tavily_search',
    'arguments': {'query1': 'latest version of gpt',
     'query2': 'most recent version of gpt',
     '

In [10]:
import json
print(search_queries['tool_results']["tavily_search"])
print(len(search_queries['tool_results']["tavily_search"]))

<search_metadata>
Queries: ['latest version of gpt', 'most recent version of gpt', 'current version of gpt']
Total Queries: 3
Max Results per Query: 15
Search Mode: Advanced
Raw Content: False
</search_metadata>

<query_section>
Query: latest version of gpt
</query_section>

<chunk 1>
Query: latest version of gpt
Title: OpenAI unveils GPT‑5. Here's what to know about the latest version ...
URL: https://www.cbsnews.com/news/openai-launches-chatgpt5-sam-altman-smartest-ai-chatbot/
Score: 0.8479807
Published: N/A
Content: The artificial intelligence company that brought the world ChatGPT says its latest version of the AI-powered chatbot will be more accurate, have fewer hallucinations and offer more articulate writing capabilities for composing emails and reports, for example. GPT-5 will also excel at coding and answering health-related questions, the company said in a blog post. A basic version of the new model is available for free, with paid options for higher usage also available. [..

## Deep Research 1


### Deep Research Clarification Asker

In [14]:
task = """You are an expert on asking clarifying Questions which will let you perform deep research on the user question. 

1. Identify 3 different questions to ask the user the answers to which will help you answer the user question.
2. keep the questions short and concise.
3. The questions should clarify what the user wants to research / learn / understand / plan out.
4. Once the user answers the questions, use their responses to guide your research and provide more tailored information.

For example, if the user asks: 
Example 1:
User Question: Help me prepare an itinerary for my trip to Japan next month.
Clarifying questions: 
1. What specific cities in Japan are you planning to visit?
2. How many days do you have for your trip?
3. What type of activities are you interested in (e.g., cultural, outdoor, culinary)?
4. What is your budget for the trip?

Example 2:
User Question: Which city are you moving to?
Clarifying Questions:
1. Which city are you moving to?
2. What’s your reason for the move (e.g., job, studies, lifestyle change)?
3. What’s your estimated move-in date?
4. Do you need help finding housing, setting up utilities, understanding the local job market, or anything else?
5. What is your budget for the move and for monthly living expenses?


Example 3:
User Question:
I want to understand Autoencoders. Help me understand the key architectures, starting from basics to good models

Clarifying Questions:
1. What is your primary purpose for learning about autoencoders? (e.g., anomaly detection, dimensionality reduction, data generation, etc.)
2. What level of detail are you looking for in the architecture explanations? Do you want theoretical grounding, mathematical formulation, and implementation-level details (e.g., PyTorch or TensorFlow)?
3. Are you interested only in standard autoencoders or also in variants like variational autoencoders (VAEs), denoising autoencoders, sparse autoencoders, etc.?
4. What is your level of current understanding? (e.g., beginner, intermediate, advanced)
"""
model1="mistral-small3.2:latest"

clarifying_agent = OllamaClient(role = task, model_name=model1, agent_name="ClarifyingAgent")

class ClarifyingQuestions(BaseModel):
    question_1: str
    question_2: str
    question_3: str


#clarifying_questions = re_phrase_agent.invoke(query="""Help me plan a birthday party""", json_schema=ClarifyingQuestions)
clarifying_questions = clarifying_agent.invoke(query="""Help me understand jung's contribution to psychology""", json_schema=ClarifyingQuestions)

clarifying_questions

{'text': '{ "question_1": "What specific aspects of Jung\'s contributions to psychology are you most interested in understanding (e.g., archetypes, collective unconscious, individuation, personality types)?" ,\n  "question_2": "Are you looking for a general overview of Jung\'s theories or a deeper exploration of his works, including his books and essays?",\n  "question_3": "How would you like the information presented (e.g., summaries, comparisons with Freud\'s theories, practical applications in modern psychology)?"}',
 'raw': {'model': 'mistral-small3.2:latest',
  'created_at': '2025-08-24T16:48:01.689460533Z',
  'message': {'role': 'assistant',
   'content': '{ "question_1": "What specific aspects of Jung\'s contributions to psychology are you most interested in understanding (e.g., archetypes, collective unconscious, individuation, personality types)?" ,\n  "question_2": "Are you looking for a general overview of Jung\'s theories or a deeper exploration of his works, including his 

In [19]:
from IPython.display import display, Markdown
Markdown(clarifying_agent.get_agent_context())

<Agent: ClarifyingAgent>
            Task to be done by the agent:
This is the You are an expert on asking clarifying Questions which will let you perform deep research on the user question. 

1. Identify 3 different questions to ask the user the answers to which will help you answer the user question.
2. keep the questions short and concise.
3. The questions should clarify what the user wants to research / learn / understand / plan out.
4. Once the user answers the questions, use their responses to guide your research and provide more tailored information.

For example, if the user asks: 
Example 1:
User Question: Help me prepare an itinerary for my trip to Japan next month.
Clarifying questions: 
1. What specific cities in Japan are you planning to visit?
2. How many days do you have for your trip?
3. What type of activities are you interested in (e.g., cultural, outdoor, culinary)?
4. What is your budget for the trip?

Example 2:
User Question: Which city are you moving to?
Clarifying Questions:
1. Which city are you moving to?
2. What’s your reason for the move (e.g., job, studies, lifestyle change)?
3. What’s your estimated move-in date?
4. Do you need help finding housing, setting up utilities, understanding the local job market, or anything else?
5. What is your budget for the move and for monthly living expenses?


Example 3:
User Question:
I want to understand Autoencoders. Help me understand the key architectures, starting from basics to good models

Clarifying Questions:
1. What is your primary purpose for learning about autoencoders? (e.g., anomaly detection, dimensionality reduction, data generation, etc.)
2. What level of detail are you looking for in the architecture explanations? Do you want theoretical grounding, mathematical formulation, and implementation-level details (e.g., PyTorch or TensorFlow)?
3. Are you interested only in standard autoencoders or also in variants like variational autoencoders (VAEs), denoising autoencoders, sparse autoencoders, etc.?
4. What is your level of current understanding? (e.g., beginner, intermediate, advanced)


            ### User Input to the agent: Help me understand jung's contribution to psychology
            ### Agent's Response: { "question_1": "What specific aspects of Jung's contributions to psychology are you most interested in understanding (e.g., archetypes, collective unconscious, individuation, personality types)?" ,
  "question_2": "Are you looking for a general overview of Jung's theories or a deeper exploration of his works, including his books and essays?",
  "question_3": "How would you like the information presented (e.g., summaries, comparisons with Freud's theories, practical applications in modern psychology)?"}
            
            </Agent: ClarifyingAgent>

### Deep Research Search Agent

In [ ]:
task = """You are an expert on observing Agent-User conversations and understanding user intent for making search tool calls. 
Observe the shared conversation between user and Agent and generate 5 questions which, when searched can answer the user's intent.
1. Identify 6 questions based on the conversation on which web search can be done.
2. keep the search questions short and concise. 
3. Invoke the tavily_search if it will help provide an updated and more accurate answer to the question.
4. If invoking the tool, extract from the agent-user question 6 different ways of formulating the same question and pass to the search tool.
5. If the question needs to refer to documentation, do a web search and answer
6. If the user asks for recent events or asks to refer to the web, then use this tool
7. Rephrase the user question in 6 ways and pass each of them to the tool
"""
model1="mistral-small3.2:latest"

deep_search_question_formulator_1 = OllamaClient(role = task, model_name=model1, agent_name="DeepSearchAgent1")

class DeepSearchAgent1(BaseModel):
    question1: str
    question2: str
    question3: str
    question4: str
    question5: str
    question6: str

search_queries = deep_search_question_formulator_1.invoke(query=f"""
{clarifying_agent.get_agent_context()}
""", tools=[tavily_search])

search_queries

{'text': "I need to observe the conversation between the user and the agent to generate relevant search questions. However, I don't see any conversation provided in the context. Could you please provide the conversation between the user and the agent so that I can proceed with generating the search questions?",
 'raw': {'model': 'mistral-small3.2:latest',
  'created_at': '2025-08-24T17:04:58.966126539Z',
  'message': {'role': 'assistant',
   'content': "I need to observe the conversation between the user and the agent to generate relevant search questions. However, I don't see any conversation provided in the context. Could you please provide the conversation between the user and the agent so that I can proceed with generating the search questions?"},
  'done_reason': 'stop',
  'done': True,
  'total_duration': 47382620911,
  'load_duration': 51243654,
  'prompt_eval_count': 804,
  'prompt_eval_duration': 38589976182,
  'eval_count': 55,
  'eval_duration': 8702397871},
 'conversation_h

In [18]:
re_phrase_agent.get_agent_context()

"<Agent: DeepSearchAgent1>\n            Task to be done by the agent:\nThis is the You are an expert on observing Agent-User conversations and understanding user intent for making search tool calls. \nObserve the shared conversation between user and Agent and generate 5 questions which, when searched can answer the user's intent.\n1. Identify 6 questions based on the conversation on which web search can be done.\n2. keep the search questions short and concise. \n3. Invoke the tavily_search if it will help provide an updated and more accurate answer to the question.\n4. If invoking the tool, extract from the agent-user question 6 different ways of formulating the same question and pass to the search tool.\n5. If the question needs to refer to documentation, do a web search and answer\n6. If the user asks for recent events or asks to refer to the web, then use this tool\n7. Rephrase the user question in 6 ways and pass each of them to the tool\n\n\n            ### User Input to the agent

### Generic Tool calling function

In [ ]:
def call_tools():
    

### Agent that summarizes the Answer

In [ ]:
task = """You are an Expert on Answering a User Question based on provided context. 
Study the context deeply and answer the question asked by the user. Use a table in the end to summarize the answer"""
model1="qwen3:1.7b"
model2="llama3.2-vision:11b"
model3="gemma3:4b"


re_phrase_agent = OllamaClient(role = task, model_name=model2)

class AnswerBasedOnContext(BaseModel):
    answer: str
    


re_phrase_agent.invoke(query="""Hey Man. i love coding. 
                       I wanted to ask you several questions one of which is: How can I improve the performance of my Python code?""", json_schema=RePhraseResult)

HTTPError: 404 Client Error: Not Found for url: http://localhost:11434/api/chat